# Métricas do reprocessamento

Código feito por Igor Laltuf, set/23

---

## Objetivo

Analisar o impacto do reprocessamento das viagens da linha 309 em 31/12/2022 e 01/01/2023.


### Carregar bibliotecas

In [3]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import folium
from datetime import timedelta, datetime
from shapely import wkt

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

### 1. Comparar a variação na km após o reprocessamento

In [4]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_completa`
WHERE
  DATA BETWEEN "2022-12-31"
  AND "2023-01-01"
  AND servico_informado = '309'
GROUP BY
  mes_ano
"""

viagens_realizadas = bd.read_sql(q, from_file=True)

viagens_realizadas.to_csv('../data/treated/viagens_realizadas.csv', index=False)

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.43rows/s]


In [5]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_completa` 
WHERE
  DATA BETWEEN "2022-12-31"
  AND "2023-01-01"
  AND servico_informado = '309'
GROUP BY
  mes_ano
"""
viagens_reprocessadas = bd.read_sql(q, from_file=True)

viagens_reprocessadas.to_csv('../data/treated/viagens_reprocessadas.csv', index=False)

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.05rows/s]


In [6]:
viagens_reprocessadas = pd.read_csv('../data/treated/viagens_reprocessadas.csv')
viagens_realizadas = pd.read_csv('../data/treated/viagens_realizadas.csv')

km_mes_ano = pd.merge(viagens_realizadas, viagens_reprocessadas, how = 'left', on='mes_ano', suffixes=['','_reprocessada'])
km_mes_ano = km_mes_ano.sort_values('mes_ano')
km_mes_ano

,mes_ano,distancia_total,distancia_total_reprocessada
1,2022-12,3658.74,7407.94
0,2023-01,1971.20,4603.58


In [7]:
distancia_total_pre_reprocessamento = km_mes_ano['distancia_total'].sum()
distancia_total_pos_reprocessamento = km_mes_ano['distancia_total_reprocessada'].sum()
print("Comparação da quilometragem das viagens completas")
print("Valor total da km pré-reprocessamento:", distancia_total_pre_reprocessamento)
print("Valor total da km pós-reprocessamento:", distancia_total_pos_reprocessamento)
print("Aumento de ",round((distancia_total_pos_reprocessamento / distancia_total_pre_reprocessamento - 1) * 100, 2),"%")

Comparação da quilometragem das viagens completas
Valor total da km pré-reprocessamento: 5629.9399999999905
Valor total da km pós-reprocessamento: 12011.520000000008
Aumento de  113.35 %


In [8]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos para km_mes_ano['distancia_total']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total'],
                         mode='lines+markers',
                         name='Km pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para km_mes_ano['distancia_total_reprocessada']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total_reprocessada'],
                         mode='lines+markers',
                         name='Km reprocessada',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Quilometragem apurada por mês - Serviço: 309", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 2. Comparar a variação no valor do subsídio após o reprocessamento

2.1 Valor recebido pelos consórcios antes do reprocessamento

In [9]:
q = f"""
SELECT
  CONCAT(CAST(EXTRACT(YEAR FROM DATA) AS STRING), '-', LPAD(CAST(EXTRACT(MONTH FROM DATA) AS STRING), 2, '0')) AS mes_ano,
  SUM(viagens) as viagens,
  
  SUM(valor_subsidio_pago) AS valor_subsidio
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  DATA BETWEEN "2022-12-31"
  AND "2023-01-01"
  AND servico = '309'
GROUP BY
  mes_ano
"""
subsidio_pago_mes = bd.read_sql(q, from_file=True)


Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.06rows/s]


In [10]:
subsidio_pago_mes = subsidio_pago_mes.sort_values('mes_ano')
subsidio_pago_mes = subsidio_pago_mes[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_pago_mes

,mes_ano,viagens,valor_subsidio
0,2022-12,102,0.0
1,2023-01,45,0.0


2.2 Valor que deve ser pago após o reprocessamento

Com base nas viagens identificadas na tabela de viagem completa que foi reprocessada, foram verificados aqueles dias em que POD >= 80%, para então calcular o valor do subsídio de acordo com os valores por km da tabela `subsidio_data_versao_efetiva` que está no dataset `rj-smtr-dev.SMTR202302003175_reprocessamento`.

Vale destacar que esta tabela exclui as glosas de 2023 do valor do subsídio por km. Ex: considera 2.81 por km após 16 de janeiro de 2023. 

In [11]:
q = f"""
WITH
  viagem_dia AS (
  SELECT
    DATA,
    COUNT(*) AS viagens,
    SUM(distancia_planejada) AS km_apurada
  FROM
    `rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_completa`
  WHERE
    DATA BETWEEN "2022-12-31"
    AND "2023-01-01"
  GROUP BY
    DATA
  ),
  viagem_planejada AS ( -- pegar a km planejada para o dia
  SELECT DISTINCT 
    DATA,
    distancia_total_planejada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
  WHERE
    servico = '309' AND
    DATA BETWEEN "2022-12-31"
    AND "2023-01-01"
  ),
  tabela_pod AS (
  SELECT
    *,
    COALESCE(ROUND((km_apurada / NULLIF(distancia_total_planejada, 0)), 2), 0) AS perc_km_planejada
  FROM
    viagem_dia vd
  RIGHT JOIN
    viagem_planejada vp
  USING
    (DATA) 
  ),
  tabela_subsidio_km AS (
  SELECT
    DATA,
    valor_subsidio_por_km
  FROM
    `rj-smtr-dev.SMTR202302003175_reprocessamento.subsidio_data_versao_efetiva` )
SELECT
  DATA,
  COALESCE(viagens, 0) as viagens,
  COALESCE(km_apurada,0) as km_apurada,
  COALESCE(distancia_total_planejada,0) as distancia_total_planejada,
  COALESCE(perc_km_planejada,0) as perc_km_planejada,
  COALESCE(valor_subsidio_por_km,0) as valor_subsidio_por_km,
   CASE 
   WHEN perc_km_planejada >= 0.8 THEN valor_subsidio_por_km * km_apurada
   ELSE 0
   END as valor_subsidio
FROM
  tabela_pod
LEFT JOIN
  tabela_subsidio_km
USING
  (DATA)
"""

subsidio_a_pagar_dia = bd.read_sql(q, from_file=True)
subsidio_a_pagar_dia = subsidio_a_pagar_dia.sort_values('DATA')
subsidio_a_pagar_dia

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.89rows/s]


,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio
1,2022-12-31,206,7408.256,7327.146,1.01,1.78,13186.69568
0,2023-01-01,128,4603.563,4889.270,0.94,3.18,14639.33034


In [12]:
subsidio_a_pagar_dia['DATA'] = pd.to_datetime(subsidio_a_pagar_dia['DATA'])
subsidio_a_pagar_dia['mes_ano'] = subsidio_a_pagar_dia['DATA'].dt.strftime('%Y-%m')
subsidio_a_pagar_mes = subsidio_a_pagar_dia[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_a_pagar_mes = subsidio_a_pagar_mes.groupby('mes_ano').sum().reset_index()  
subsidio_a_pagar_mes

,mes_ano,viagens,valor_subsidio
0,2022-12,206,13186.69568
1,2023-01,128,14639.33034


In [13]:
# subsidio pago por dia

q = f"""
WITH
  viagem_dia AS (
  SELECT
    DATA,
    COUNT(*) AS viagens,
    SUM(distancia_planejada) AS km_apurada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_completa`
  WHERE
    servico_informado = '309' 
    AND DATA BETWEEN "2022-12-31"
    AND "2023-01-01"
  GROUP BY
    DATA
  ),
  viagem_planejada AS ( -- pegar a km planejada para o dia
  SELECT DISTINCT 
    DATA,
    distancia_total_planejada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
  WHERE
    servico = '309' AND
    DATA BETWEEN "2022-12-31"
    AND "2023-01-01"
  ),
  tabela_pod AS (
  SELECT
    *,
    COALESCE(ROUND((km_apurada / NULLIF(distancia_total_planejada, 0)), 2), 0) AS perc_km_planejada
  FROM
    viagem_dia vd
  RIGHT JOIN
    viagem_planejada vp
  USING
    (DATA) 
  ),
  tabela_subsidio_km AS (
  SELECT
    DATA,
    valor_subsidio_por_km
  FROM
    `rj-smtr.projeto_subsidio_sppo.subsidio_data_versao_efetiva` )
SELECT
  DATA,
  COALESCE(viagens, 0) as viagens,
  COALESCE(km_apurada,0) as km_apurada,
  COALESCE(distancia_total_planejada,0) as distancia_total_planejada,
  COALESCE(perc_km_planejada,0) as perc_km_planejada,
  COALESCE(valor_subsidio_por_km,0) as valor_subsidio_por_km,
   CASE 
   WHEN perc_km_planejada >= 0.8 THEN valor_subsidio_por_km * km_apurada
   ELSE 0
   END as valor_subsidio
FROM
  tabela_pod
LEFT JOIN
  tabela_subsidio_km
USING
  (DATA)
"""

subsidio_pago_dia = bd.read_sql(q, from_file=True)
subsidio_pago_dia = subsidio_a_pagar_dia.sort_values('DATA')
subsidio_pago_dia

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.80rows/s]


,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio,mes_ano
1,2022-12-31,206,7408.256,7327.146,1.01,1.78,13186.69568,2022-12
0,2023-01-01,128,4603.563,4889.270,0.94,3.18,14639.33034,2023-01


In [14]:
subsidio_a_pagar_dia = subsidio_a_pagar_dia.drop(['mes_ano'], axis=1)
subsidio_a_pagar_dia

,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio
1,2022-12-31,206,7408.256,7327.146,1.01,1.78,13186.69568
0,2023-01-01,128,4603.563,4889.270,0.94,3.18,14639.33034


O aumento do valor do subsídio não é proporcional ao aumento da quantidade de viagens, pois com o reprocessamento, cada viagem circular virou duas viagens separadas, uma de ida e outra de volta.

Isto quer dizer que o esperado era que a quantidade de viagens pelo menos dobrasse após o reprocessamento, mesmo que não fosse identificada nenhuma viagem nova.

Valor do subsídio a ser pago por dia após o reprocessamento

In [15]:
q = f"""
SELECT
  *
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '309'
  AND DATA BETWEEN '2022-12-31' 
  AND '2023-01-01'
"""
subsidio_pago_dia = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.52rows/s]


In [16]:
subsidio_pago_dia = subsidio_pago_dia[['data', 'viagens', 'valor_subsidio_pago']]
subsidio_pago_dia

subsidio_pago_dia = subsidio_pago_dia.rename(columns={
    'viagens': 'viagens_pre',
    'valor_subsidio_pago': 'subsidio_pre'
})

subsidio_pago_dia['subsidio_pre'] = subsidio_pago_dia['subsidio_pre'].round(2)

subsidio_pago_dia


,data,viagens_pre,subsidio_pre
0,2023-01-01,45,0.0
1,2022-12-31,102,0.0


In [17]:
subsidio_a_pagar_dia_formatado = subsidio_a_pagar_dia[['DATA', 'viagens', 'valor_subsidio']]

subsidio_a_pagar_dia_formatado = subsidio_a_pagar_dia_formatado.rename(columns={
    'DATA': 'data',
    'viagens': 'viagens_pos',
    'valor_subsidio': 'subsidio_pos'
})

subsidio_a_pagar_dia_formatado['subsidio_pos'] = subsidio_a_pagar_dia_formatado['subsidio_pos'].round(2)

# Certificando que 'data' é do tipo datetime em ambos os DataFrames
subsidio_a_pagar_dia_formatado['data'] = pd.to_datetime(subsidio_a_pagar_dia_formatado['data'])

subsidio_pago_dia['data'] = pd.to_datetime(subsidio_pago_dia['data'])

resultado = pd.merge(subsidio_a_pagar_dia_formatado, subsidio_pago_dia, on='data', how='outer')

resultado['subsidio_a_pagar'] = resultado['subsidio_pos'] - resultado['subsidio_pre']

resultado.to_excel('./../data/output/subsidio_a_pagar_dia.xlsx', index=False)

In [18]:
# Aqui o comparativo deve ser por dia e não por mês!!!!
# 1.78 e 3.18
resultado

,data,viagens_pos,subsidio_pos,viagens_pre,subsidio_pre,subsidio_a_pagar
0,2022-12-31,206,13186.70,102,0.0,13186.70
1,2023-01-01,128,14639.33,45,0.0,14639.33


In [19]:
# juntar as duas tabelas
tabela_subsidio_mes = pd.merge(subsidio_pago_mes, subsidio_a_pagar_mes, how='left', on='mes_ano', suffixes=['_pre','_pos'])
tabela_subsidio_mes

,mes_ano,viagens_pre,valor_subsidio_pre,viagens_pos,valor_subsidio_pos
0,2022-12,102,0.0,206,13186.69568
1,2023-01,45,0.0,128,14639.33034


In [20]:
tabela_subsidio_mes
total_subsidio_comparacao = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre','valor_subsidio_pos']].sum(axis=0)
total_subsidio_comparacao

viagens_pre             147.00000
viagens_pos             334.00000
valor_subsidio_pre        0.00000
valor_subsidio_pos    27826.02602
dtype: float64